In [ ]:
'''
loading libs
'''
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
'''
loading images
'''
nevus_filepath = 'data/0/'
melanoma_filepath = 'data/1/'
# get dot filenames
f = open('n_dots.txt')
data = f.read()
f.close()
n_dots_filenames = data.split('\n')

nevus_files = [f for f in os.listdir(nevus_filepath) if os.path.isfile(nevus_filepath+f)]
melanoma_files = [f for f in os.listdir(melanoma_filepath) if os.path.isfile(melanoma_filepath+f)]
# sort filenames
nevus_files = sorted(nevus_files)
melanoma_files = sorted(melanoma_files)

N = 1000
# Load Nevus
nevus_imgs=[]
n_dots_imgs=[]
for i in range(N):
    if nevus_files[i] not in n_dots_filenames:
        nevus_imgs += [plt.imread(os.path.join(nevus_filepath, nevus_files[i]))]
    else:
        n_dots_imgs += [plt.imread(os.path.join(nevus_filepath, nevus_files[i]))]

melanoma_imgs = [plt.imread(os.path.join(melanoma_filepath, melanoma_files[i])) for i in range(N)]

# resize
nevus_imgs = [cv2.resize(img,(256,256)) for img in nevus_imgs]
melanoma_imgs = [cv2.resize(img,(256,256)) for img in melanoma_imgs]
n_dots_imgs = [cv2.resize(img,(256,256)) for img in n_dots_imgs]

In [ ]:
'''
Split data
'''
from sklearn.model_selection import train_test_split
X = nevus_imgs + n_dots_imgs

X = np.array([img.reshape(256,256,1) for img in X])
y = np.zeros(X.shape[0])
y[len(nevus_imgs):] = 1

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [ ]:
'''
Define plot training function
'''
import matplotlib.pyplot as plt

def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)
    plt.plot(epochs, acc, label='Training acc')
    plt.plot(epochs, val_acc, label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epochs')
    plt.legend()

    plt.figure()
    plt.plot(epochs, loss, label='Training loss')
    plt.plot(epochs, val_loss, label='Validation loss')
    plt.title('Training and validation loss')
    plt.ylabel('Loss')
    plt.xlabel('Epochs')
    plt.legend()
    plt.show()

In [ ]:
import tensorflow

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

print(tensorflow.__version__)
print("Num GPUs Available: ", len(tensorflow.config.experimental.list_physical_devices('GPU')))

tensorflow.debugging.set_log_device_placement(True)
gpus = tensorflow.config.experimental.list_physical_devices('GPU')
tensorflow.config.experimental.set_memory_growth(gpus[0], True)

In [ ]:
# My model
model = models.Sequential()
model.add(layers.Conv2D(32, (4,4), activation='relu', input_shape=(256,256,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
# model.add(layers.Dropout(0.3))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
     
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs = 50,
    validation_split=0.2)

In [ ]:
plot_history(history)
test_loss, test_acc = model.evaluate(X_test, y_test)
print("test acc:", test_acc)
print("test loss:", test_loss)

In [ ]:
'''
Apply model to real data
'''
data = np.array([img.reshape(256,256,1) for img in melanoma_imgs]) # nevus_imgs
n_classes = model.predict(data)
n_classes
dots_classes = n_classes > 0.4

fig, ax = plt.subplots(15,15, figsize=(15,15))
k = 0
for i in range(15):
    for j in range(15):
        if dots_classes[k]:
            ax[i,j].imshow(melanoma_imgs[k], 'Greens_r')
        else:
            ax[i,j].imshow(melanoma_imgs[k], 'Blues_r')
        ax[i,j].axis('off')
        k+=1
        
plt.figure()
plt.plot(dots_classes, '--r', alpha=0.4)
plt.plot(n_classes, 'b')